# Motor de Investigación con LCEL

## Instalación de paquetes
Si estás corriendo este notebook en Google Colab, corre la siguiente celda para instalar los paquetes necesarios.

In [1]:
# %pip install langchain langchain_community langchain_openai duckduckgo-search


In [2]:
# Corre esta celda solo si tienes un archivo .env configurado
from dotenv import load_dotenv
from duckduckgo_search.utils import json_loads

load_dotenv()

True

Para quienes desarrollan aplicaciones con LLM, el uso de LCEL es altamente recomendado. Este permite interactuar con los LLM y modelos de chat de manera eficiente mediante la creación y ejecución de cadenas, proporcionando varios beneficios:

- **Fallback**: Permite agregar una acción de respaldo para el manejo de errores.
- **Ejecución en paralelo**: Ejecuta componentes de las cadenas de forma independiente y simultánea para mejorar el rendimiento.
- **Modos de ejecución**: Soporta el desarrollo en modo síncrono y luego permite cambiar a modos de ejecución por streaming, por lotes o asincrónicos, según sea necesario.
- **Rastreo con LangSmith**: Registra automáticamente los pasos de ejecución al actualizar a LangSmith, facilitando la depuración y el monitoreo.

Una cadena sigue el protocolo `Runnable`, lo que significa que requiere la implementación de métodos específicos como `invoke()`, `stream()`, y `batch()`, incluidas sus versiones asincrónicas. El framework de LangChain asegura que sus componentes, como `PromptTemplate` y `JsonOutputFunctionsParser`, cumplan con estos estándares.

Para obtener información detallada y ejemplos, te recomiendo visitar la documentación oficial de LCEL, especialmente las secciones de "How To" y "Cookbook": https://python.langchain.com/docs/expression_language/

## Arquitectura del motor de investigación con LCEL

Básicamente, vamos reimplementar cada paso del motor de investigación como una minicadena usando LCEL. Luego estas minicadenas se ensamblarán en una gran cadena. <br /><br />

Las minicadenas serán las siguientes:<br /><br />

1. Cadena de Instrucciones de Asistente
2. Cadena de Búsqueda en la Web
3. Cadena de Búsqueda y Resumen
4. Cadena de Reporte Final de Investigación
<br /><br />
Las 4 minicadenas componen la **Cadena de Investigación Web**.

In [3]:
# Reutilizamos la plantilla de instrucciones de asistente

from langchain.prompts import PromptTemplate

instrucciones_asistente_seleccion = """
Eres experto en asignar una pregunta de investigación al asistente de investigación correcto.
Hay varios asistentes de investigación disponibles, cada uno especializado en un área de experiencia.
Cada asistente está identificado por un tipo específico y tiene instrucciones específicas para llevar a cabo la investigación.

Cómo seleccionar el asistente correcto: debes seleccionar el asistente relevante dependiendo del tema de la pregunta, que debe coincidir con el área de experiencia del asistente.

------
Aquí tienes algunos ejemplos de cómo devolver la información correcta del asistente, según la pregunta realizada.

Ejemplos:
Pregunta: "¿Debería invertir en títulos de cobertura?"
Respuesta: 
{{
    "tipo_asistente": "Asistente analista financiero",
    "instrucciones_asistente": "Eres un asistente de inteligencia artificial experto en análisis financiero. Tu objetivo principal es elaborar informes financieros completos, perspicaces, imparciales y organizados metódicamente basados en los datos y tendencias proporcionados.",
    "pregunta_usuario": {pregunta_usuario}
}}
Pregunta: "¿Cuáles son los sitios más interesantes en Río de Janeiro?"
Respuesta: 
{{
    "tipo_asistente": "Asistente de guía turístico",
    "instrucciones_asistente": "Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.",
    "pregunta_usuario": "{pregunta_usuario}"
}}

Pregunta: "¿Es Messi un buen jugador de fútbol?"
Respuesta: 
{{
    "tipo_asistente": "Asistente experto en deportes",
    "instrucciones_asistente": "Eres un asistente de inteligencia artificial especializado en deportes. Tu propósito principal es redactar informes deportivos atractivos, perspicaces, imparciales y bien estructurados sobre personalidades deportivas o eventos deportivos, incluyendo detalles factuales, estadísticas y análisis.",
    "pregunta_usuario": "{pregunta_usuario}"
}}

------
Ahora que has entendido todo lo anterior, selecciona el asistente de investigación correcto para la siguiente pregunta.
Pregunta: {pregunta_usuario}
Respuesta:

""" 

plantilla_asistente_seleccion = PromptTemplate.from_template(
    template=instrucciones_asistente_seleccion
)

In [4]:
# Instanciamos el LLM
import os
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=os.getenv("MODEL"),
    openai_api_key=os.getenv("LIA_API_KEY"),
    openai_api_base=os.getenv("LIA_API_BASE"),
    temperature=0.6,
)

### 1. Cadena de Instrucciones de Asistente

In [5]:
# Ahora construimos la cadena de instrucciones de asistente
from langchain.schema.output_parser import StrOutputParser
import json
 
cadena_instrucciones_asistente = (
    plantilla_asistente_seleccion |  llm | StrOutputParser()
)

In [6]:
# Hacemos una prueba rápida
respuesta_llm = cadena_instrucciones_asistente.invoke("Qué puedo hacer en la Isla de Margarita?")

respuesta_llm

'{\n    "tipo_asistente": "Asistente de guía turístico",\n    "instrucciones_asistente": "Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.",\n    "pregunta_usuario": "Qué puedo hacer en la Isla de Margarita?"\n}'

In [7]:
# Cadena mejorada

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.runnable import RunnableLambda


cadena_instrucciones_asistente = (
    {'pregunta_usuario': RunnablePassthrough()} |
    plantilla_asistente_seleccion |  llm | StrOutputParser() | RunnableLambda(json.loads)
)

In [8]:
respuesta_llm = cadena_instrucciones_asistente.invoke("Qué puedo hacer en la Isla de Margarita?")

respuesta_llm

{'tipo_asistente': 'Asistente de guía turístico',
 'instrucciones_asistente': 'Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.',
 'pregunta_usuario': 'Qué puedo hacer en la Isla de Margarita?'}

In [9]:
# Verifiquemos si respuesta_llm es un diccionario
print(type(respuesta_llm))

<class 'dict'>


### 2. Cadena de Búsqueda en la Web

In [10]:
instrucciones_busqueda_web = """
{instrucciones_asistente}

Escribe {num_consultas_busqueda} consultas de búsqueda web para recopilar la mayor cantidad de información posible 
sobre la siguiente pregunta: {pregunta_usuario}. Tu objetivo es escribir un informe basado en la información que encuentres.
Debes responder con una lista de consultas como consulta1, consulta2, consulta3 en el siguiente formato: 
[
    {{"consulta_busqueda": "consulta1", "pregunta_usuario": "{pregunta_usuario}" }},
    {{"consulta_busqueda": "consulta2", "pregunta_usuario": "{pregunta_usuario}" }},
    {{"consulta_busqueda": "consulta3", "pregunta_usuario": "{pregunta_usuario}" }}
]
"""

plantilla_busqueda_web = PromptTemplate.from_template(
    template=instrucciones_busqueda_web
)

In [11]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
 
NUMERO_CONSULTAS_BUSQUEDA = 3
 
cadena_busqueda_web = (
    RunnableLambda(lambda x:
        {
            'instrucciones_asistente': x['instrucciones_asistente'],
            'num_consultas_busqueda': NUMERO_CONSULTAS_BUSQUEDA,
            'pregunta_usuario': x['pregunta_usuario']
        }
    )
    | plantilla_busqueda_web | llm | StrOutputParser() | RunnableLambda(json.loads)
)

In [12]:
# Hagamos una prueba de la nueva cadena

string_instruccion_asistente = '{\n    "tipo_asistente": "Asistente de guía turístico",\n    "instrucciones_asistente": "Eres un asistente de guía turístico con experiencia global. Tu propósito principal es redactar informes de viaje atractivos, perspicaces, imparciales y bien estructurados sobre ubicaciones específicas, incluyendo historia, atracciones y conocimientos culturales.",\n    "pregunta_usuario": "Qué puedo hacer en la Isla de Margarita?"\n}'

dict_instruccion_asistente = json.loads(string_instruccion_asistente)
lista_busquedas_web = cadena_busqueda_web.invoke(dict_instruccion_asistente)
 
lista_busquedas_web


[{'consulta_busqueda': 'actividades turísticas Isla de Margarita',
  'pregunta_usuario': 'Qué puedo hacer en la Isla de Margarita?'},
 {'consulta_busqueda': 'atracciones y lugares de interés Isla de Margarita',
  'pregunta_usuario': 'Qué puedo hacer en la Isla de Margarita?'},
 {'consulta_busqueda': 'cultura y costumbres Isla de Margarita',
  'pregunta_usuario': 'Qué puedo hacer en la Isla de Margarita?'}]

### 3. Cadena de Búsqueda y Resumen
<br /> <br />
Como es más complicadas, la vamos a dividir en subcadenas más pequeñas:
<br /> <br />
- Subcadena de URLs de resultados de búsqueda
- Subcadena de Texto y Resumen de los resultados de búsqueda
- Subcadena de Búsqueda y Resumen


In [13]:
# Función de búsqueda en la web
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from typing import List
 
def web_search(web_query: str, num_results: int) -> List[str]:
    return [r["link"] for r in DuckDuckGoSearchAPIWrapper().results(web_query, num_results)]

***3.1 Subcadena de URLs de resultados de búsqueda***

In [14]:
from langchain.schema.runnable import RunnableLambda
 
NUM_RESULTADOS_POR_CONSULTA = 3
 
cadena_urls_resultados_busquedas = (
    RunnableLambda(lambda x: 
        [
            {
                'url_resultado': url, 
                'consulta_busqueda': x['consulta_busqueda'],
                'pregunta_usuario': x['pregunta_usuario']
            }
            for url in web_search(web_query=x['consulta_busqueda'], 
                                  num_results=NUM_RESULTADOS_POR_CONSULTA)
        ]
    )
)

In [15]:
# Vamos a probar la cadena de URLs de resultados de búsqueda
str_busqueda_web = '{"consulta_busqueda": "historias y cultura de la Isla de Margarita atracciones recomendadas", "pregunta_usuario": "Qué puedo hacer en la Isla de Margarita?"}'

dict_busqueda_web = json.loads(str_busqueda_web)

lista_urls_resultados_busquedas = cadena_urls_resultados_busquedas.invoke(dict_busqueda_web)

lista_urls_resultados_busquedas

[{'url_resultado': 'https://oistevos.com/isla-margarita/',
  'consulta_busqueda': 'historias y cultura de la Isla de Margarita atracciones recomendadas',
  'pregunta_usuario': 'Qué puedo hacer en la Isla de Margarita?'},
 {'url_resultado': 'https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/',
  'consulta_busqueda': 'historias y cultura de la Isla de Margarita atracciones recomendadas',
  'pregunta_usuario': 'Qué puedo hacer en la Isla de Margarita?'},
 {'url_resultado': 'https://www.lugaresturisticos.org/atractivos-turisticos-de-la-isla-de-margarita/',
  'consulta_busqueda': 'historias y cultura de la Isla de Margarita atracciones recomendadas',
  'pregunta_usuario': 'Qué puedo hacer en la Isla de Margarita?'}]

***3.2 Subcadena de Texto y Resumen de los resultados de búsqueda***

In [16]:
# Herramienta de web scraping

import requests
from bs4 import BeautifulSoup
 
def web_scrape(url: str) -> str:
    try:
        response = requests.get(url)
 
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
            page_text = soup.get_text(separator=" ", strip=True)
 
            return page_text
        else:
            return f"No se pudo recuperar la página web: Código {response.status_code}"
    except Exception as e:
        print(e)
        return f"No se pudo recuperar la página web: {e}"

In [17]:
instrucciones_resumen = """
Lee el siguiente texto:
Texto: {texto_resultado_busqueda}

-----------

Usando el texto anterior, responde brevemente la siguiente pregunta.
Pregunta: {consulta_busqueda}

Si no puedes responder la pregunta anterior usando el texto proporcionado, simplemente resume el texto.

Incluye toda la información factual, números, estadísticas, etc., si están disponibles.
"""

plantilla_resumen = PromptTemplate.from_template(
    template=instrucciones_resumen
)

In [18]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableParallel

 
MAXIMO_CARACTERES_TEXTO_RESULTADO = 10000
 
cadena_resumen_busqueda = (
    RunnableLambda(lambda x:
        {
            'texto_resultado_busqueda': web_scrape(url=x['url_resultado'])[:MAXIMO_CARACTERES_TEXTO_RESULTADO],
            'url_resultado': x['url_resultado'], 
            'consulta_busqueda': x['consulta_busqueda'],
            'pregunta_usuario': x['pregunta_usuario']
        }
    )
    | RunnableParallel (
        {
            'resumen_texto': plantilla_resumen | llm | StrOutputParser(),
            'url_resultado': lambda x: x['url_resultado'],
            'pregunta_usuario': lambda x: x['pregunta_usuario']            
        }
    )
    | RunnableLambda(lambda x: 
        {
            'resumen': f"Fuente Url: {x['url_resultado']}\nResumen: {x['resumen_texto']}",
            'pregunta_usuario': x['pregunta_usuario']
        }
    ) 
)

In [19]:
# Vamos a probar la cadena de texto y resumen de los resultados de búsqueda

dict_url_resultado = lista_urls_resultados_busquedas[0]

resumen_resultado_busqueda = cadena_resumen_busqueda.invoke(dict_url_resultado)

resumen_resultado_busqueda

{'resumen': 'Fuente Url: https://oistevos.com/isla-margarita/\nResumen: La Isla Margarita, conocida como la Perla del Caribe, tiene una rica historia que se remonta a la época precolombina, habitada por los indígenas guaiqueríes. Con la llegada de Cristóbal Colón en 1498, la isla se convirtió en un centro de comercio y defensa. Durante la época colonial, la isla fue escenario de enfrentamientos entre españoles y piratas, lo que ha dejado un legado cultural que se refleja en su arquitectura y tradiciones.\n\nEntre las atracciones recomendadas para explorar su historia y cultura se encuentran los pueblos de La Asunción y Juan Griego, que son destinos imperdibles para sumergirse en la historia colonial de la isla. Además, Isla Margarita ofrece una variedad de actividades, como excursiones y deportes acuáticos en lugares como el Parque Nacional Los Frailes. Para quienes buscan un ambiente más relajado, hay opciones de playas tranquilas como Playa Manzanillo.\n\nIsla Margarita es un destino

In [20]:
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
 
cadena_busqueda_resumen = (
    cadena_urls_resultados_busquedas 
    | cadena_resumen_busqueda.map() # parallelize for each url
    | RunnableLambda(lambda x: 
        {
            'resumen': '\n'.join([i['resumen'] for i in x]), 
            'pregunta_usuario': x[0]['pregunta_usuario'] if len(x) > 0 else ''
        })
)

In [21]:
# Vamos a probar la cadena de búsqueda y resumen
cadena_busqueda_resumen.invoke(dict_busqueda_web)

{'resumen': 'Fuente Url: https://hospedamargarita.com/descubre-la-isla-de-margarita-historia-cultura-y-recreacion-en-el-caribe-venezolano/\nResumen: No se puede responder a la pregunta sobre las historias, cultura y atracciones recomendadas de la Isla de Margarita utilizando el texto proporcionado, ya que este solo informa sobre un error de acceso a una página web (Código 403). \n\nResumen del texto: No se pudo recuperar la página web debido a un error de acceso, indicado por el Código 403.\nFuente Url: https://www.01centralamerica.com/america_central/turismo/isla-margarita-venezuela/\nResumen: La Isla Margarita, ubicada al noreste de Venezuela, fue descubierta en 1498 por Cristóbal Colón y ha sido influenciada por diversas culturas, incluyendo la española, africana y aborigen. Esta mezcla cultural se refleja en su arquitectura colonial, tradiciones y gastronomía. \n\nEntre las atracciones recomendadas se encuentran:\n\n1. **Playa El Agua**: Con aproximadamente 4 kilómetros de arena do

### 4. Cadena de Reporte Final de Investigación

In [22]:
 #Prompt de informe de investigación adaptado de https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py

instrucciones_informe_investigacion = """
Eres un asistente de investigación de pensamiento crítico impulsado por IA. Tu único propósito es escribir informes bien redactados, aclamados críticamente, objetivos y estructurados sobre el texto proporcionado.

Información: 
--------
{resumen_investigacion}
--------

Usando la información anterior, responde la siguiente pregunta o tema: "{pregunta_usuario}" en un informe detallado -- \
El informe debe centrarse en la respuesta a la pregunta, estar bien estructurado, ser informativo, \
profundo, con hechos y cifras si están disponibles, y tener un mínimo de 1,200 palabras.

Debes esforzarte por escribir el informe lo más extenso posible utilizando toda la información relevante y necesaria proporcionada.
Debes escribir el informe con sintaxis de markdown.
DEBES determinar tu propia opinión concreta y válida basada en la información proporcionada. NO infieras conclusiones generales y sin sentido.
Escribe todas las URLs de las fuentes utilizadas al final del informe y asegúrate de no agregar fuentes duplicadas, solo una referencia por cada una.
Debes escribir el informe en formato APA."""
 
plantilla_informe_investigacion = PromptTemplate.from_template(
    template=instrucciones_informe_investigacion
)

In [23]:
cadena_investigacion_web = (
    cadena_instrucciones_asistente 
    | cadena_busqueda_web 
    | cadena_busqueda_resumen.map() # corre cada consulta en paralelo
    | RunnableLambda(lambda x:
       {
           'resumen_investigacion': '\n\n'.join([i['resumen'] for i in x]),
           'pregunta_usuario': x[0]['pregunta_usuario'] if len(x) > 0 else ''
        })
    | plantilla_informe_investigacion | llm | StrOutputParser()
)

In [24]:
# Vamos a probar la cadena final

reporte_investigacion = cadena_investigacion_web.invoke("Qué puedo hacer en la Isla de Margarita?")

reporte_investigacion

'# Qué puedo hacer en la Isla de Margarita\n\nLa Isla Margarita, ubicada en el estado de Nueva Esparta, Venezuela, es un destino turístico conocido por su belleza natural, rica historia y vibrante cultura. Con una superficie aproximada de 1,020 km² y un clima cálido durante todo el año, la isla ofrece una amplia variedad de actividades y atracciones para todos los gustos. A continuación, se presenta un informe detallado sobre las diversas actividades que los visitantes pueden disfrutar en este paraíso caribeño.\n\n## 1. Disfrutar de las Playas\n\nLas playas de Isla Margarita son uno de sus principales atractivos. Con más de 50 playas a lo largo de su costa, cada una ofrece una experiencia única:\n\n### 1.1 Playa El Agua\n\nPlaya El Agua es quizás la más famosa de la isla, extendiéndose por 4 kilómetros de arena blanca y aguas cristalinas. Es un lugar ideal para relajarse, tomar el sol o practicar deportes acuáticos como el windsurf y el kitesurf. A lo largo de la playa, los visitantes 